In [1]:
%matplotlib inline
import matplotlib.pylab as plt
import matplotlib as mpl
import pandas as pd
import pandas.io.sql as pandasql
import datetime
from psycopg2 import connect

In [31]:
#Setting up postgresql connection
con = connect(database='rdumas',host='localhost',port='5433',user='rdumas')

In [3]:
#Ostensibly prevents plotting when using the save_fig command, but I've seen no evidence of success
#http://stackoverflow.com/a/15713545/4047679
plt.ioff()
plt.style.use('ggplot')

In [4]:
'''
Setting colours
http://stackoverflow.com/a/37211181/4047679
Blue: #004B85
Red: #F2756D
'''
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=['#004B85','#F2756D'])

In [32]:
sql = "SELECT dt_week::date, roadname as corridor, direction, period,tt_med as median_tt "\
      "FROM rdumas.signal_timing_perf_alldata "\
      "INNER JOIN rdumas.corridors_signal_timing_lookup USING (signal_timing_id) "\
      "WHERE daytype = 'Midweek' and period IN ('AMPK','PMPK') "\
      "ORDER BY roadname, direction, period, dt_week"
signals_perf = pandasql.read_sql(sql, con)

In [27]:
dirs = signals_perf.direction.unique()
peaks = signals_perf.period.unique()
corridors = pandasql.read_sql("SELECT roadname as corridor, direction FROM corridors_signal_timing_lookup", con)

In [15]:
#Signal retiming periods
retiming_dates = {'Markham Rd':dict(x1=datetime.date(2014,8,11), x2=datetime.date(2014,8,20), text='Signal\nRetiming'),
                  'Sheppard Ave':dict(x1=datetime.date(2014,7,22), x2=datetime.date(2014,7,25), text='Signal\nRetiming')}

In [43]:
'''Median Travel Time Plots'''
ylim=[0,40]
for corridor in signals_perf.corridor.unique():
    fig, ax = plt.subplots(1,2, figsize=(16,4), sharey=True)
    
    for ax_c, pk in enumerate(peaks):
        
        for direction in corridors[corridors["corridor"]==corridor].direction.sort_values():
            signals_perf[(signals_perf["corridor"]==corridor)
                               & (signals_perf["direction"]==direction)
                               & (signals_perf["period"]==pk)].plot(x='dt_week',
                                                                          y='median_tt',
                                                                          linewidth=2,
                                                                          ax=ax[ax_c],
                                                                          label=direction
                                                                         )
        ax[ax_c].set_title(pk)
        ax[ax_c].set_autoscaley_on(False)
        ax[ax_c].set_ylim(ylim)
        ax[ax_c].axvspan(retiming_dates[corridor]['x1'], retiming_dates[corridor]['x2'], alpha=0.5, label=retiming_dates[corridor]['text'])
        ax[ax_c].set_xlabel('Month')
        ax[ax_c].xaxis.set_major_locator(mpl.dates.MonthLocator(bymonth=range(1,13,3)))
    fig.suptitle(corridor, fontsize=16)
    ax[0].set_ylabel('Median Travel Time (min)')
    ax[0].legend().set_visible(False)
    leg = ax[1].legend(loc='center right',bbox_to_anchor=(1.3,0.5))
    for legobj in leg.legendHandles:
        legobj.set_linewidth(6.0)
    plt.gcf().autofmt_xdate()
    plt.minorticks_off()
    fig.savefig('corridors/signal_retiming/median_tt_plots/'+ corridor +'.png', format='png', transparent=True)
    plt.close(fig)

## Score 30 data only

In [44]:
sql = "SELECT dt_week::date, roadname as corridor, direction, period,tt_med as median_tt "\
      "FROM rdumas.signal_timing_perf_score30 "\
      "INNER JOIN rdumas.corridors_signal_timing_lookup USING (signal_timing_id) "\
      "WHERE daytype = 'Midweek' and period IN ('AMPK','PMPK') "\
      "AND dt_week < '2016-01-01'::date "\
      "ORDER BY roadname, direction, period, dt_week"
signals_perf = pandasql.read_sql(sql, con)

In [45]:
dirs = signals_perf.direction.unique()
peaks = signals_perf.period.unique()
corridors = pandasql.read_sql("SELECT roadname as corridor, direction FROM corridors_signal_timing_lookup", con)

In [46]:
#Signal retiming periods
retiming_dates = {'Markham Rd':dict(x1=datetime.date(2014,8,11), x2=datetime.date(2014,8,20), text='Signal\nRetiming'),
                  'Sheppard Ave':dict(x1=datetime.date(2014,7,22), x2=datetime.date(2014,7,25), text='Signal\nRetiming')}

In [47]:
'''Median Travel Time Plots'''
ylim=[0,40]
for corridor in signals_perf.corridor.unique():
    fig, ax = plt.subplots(1,2, figsize=(16,4), sharey=True)
    
    for ax_c, pk in enumerate(peaks):
        
        for direction in corridors[corridors["corridor"]==corridor].direction.sort_values():
            signals_perf[(signals_perf["corridor"]==corridor)
                               & (signals_perf["direction"]==direction)
                               & (signals_perf["period"]==pk)].plot(x='dt_week',
                                                                          y='median_tt',
                                                                          linewidth=2,
                                                                          ax=ax[ax_c],
                                                                          label=direction
                                                                         )
        ax[ax_c].set_title(pk)
        ax[ax_c].set_autoscaley_on(False)
        ax[ax_c].set_ylim(ylim)
        ax[ax_c].axvspan(retiming_dates[corridor]['x1'], retiming_dates[corridor]['x2'], alpha=0.5, label=retiming_dates[corridor]['text'])
        ax[ax_c].set_xlabel('Month')
        ax[ax_c].xaxis.set_major_locator(mpl.dates.MonthLocator(bymonth=range(1,13,3)))
    fig.suptitle(corridor, fontsize=16)
    ax[0].set_ylabel('Median Travel Time (min)')
    ax[0].legend().set_visible(False)
    leg = ax[1].legend(loc='center right',bbox_to_anchor=(1.3,0.5))
    for legobj in leg.legendHandles:
        legobj.set_linewidth(6.0)
    plt.gcf().autofmt_xdate()
    plt.minorticks_off()
    fig.savefig('corridors/signal_retiming/median_tt_plots/'+ corridor +'score30.png', format='png', transparent=True)
    plt.close(fig)